In [1]:
import pandas as pd
import numpy as np

In [2]:
# Cargar Datos
df_historico = pd.read_csv('C:/Users/Nancy Sorroza Rojas/Desktop/Pool/Proyectos Data Science/WorldCup_Torunament_2026/Data/fifa_worldcup_historical_data.csv', delimiter=',')
df_fixture = pd.read_csv('C:/Users/Nancy Sorroza Rojas/Desktop/Pool/Proyectos Data Science/WorldCup_Torunament_2026/Data/fifa_worldcup_fixture.csv', delimiter=',')

In [3]:
print('Dimension of df_historico',df_historico.shape)
print('Dimension of df_fixture',df_historico.shape)
df_historico.head()

Dimension of df_historico (929, 4)
Dimension of df_fixture (929, 4)


,home,score,away,year
0,France,4–1,Mexico,1930
1,Argentina,1–0,France,1930
2,Chile,3–0,Mexico,1930
3,Chile,1–0,France,1930
4,Argentina,6–3,Mexico,1930


In [4]:
df_fixture.head()

,home,score,away,year
0,Mexico,Match 1,A2,2026
1,A3,Match 2,A4,2026
2,A4,Match 25,A2,2026
3,Mexico,Match 28,A3,2026
4,A4,Match 53,Mexico,2026
